# 8) BAC response of models

This notebook evaluates qualitatively the capability of fitted models to reproduce BAC behavior.

In [ ]:
import json
import sys
import os
import time
import neuron
from pathlib import Path
import pickle
import pandas as pd

import matplotlib.pyplot as plt
import neuroplotlib as nplt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import LFPy
import neuron
import numpy as np

import shutil

import multimodalfitting as mf

%matplotlib notebook
# %matplotlib widget

In [ ]:
base_dir = Path("../..")

In [ ]:
colors_dict = {"soma": "C0",
               "all": "C1",
               "sections": "C2",
               "single": "C3"}
figsize = (10, 7)

## Load results and define model

In [ ]:
probe_type = "planar"
model_name = "hay_ais" # "hay", "hay_ais", "hay_ais_hillock"
cell_models_folder = base_dir / "cell_models"

model_folder = cell_models_folder / f"{model_name}"

In [ ]:
result_dates = [r.name for r in (base_dir / "results").iterdir()]
# use latest results
results_date = max(result_dates)
result_folder = base_dir / "results" / results_date

In [ ]:
pkl_file_name = "runs.pkl"

data = pickle.load(open(result_folder / pkl_file_name, 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == '{model_name}'")

if (result_folder / f"all_responses_{model_name}.pkl").is_file():
    with open(result_folder / f"all_responses_{model_name}.pkl", 'rb') as f:
        opt_results_all = pickle.load(f)

In [ ]:
cell = mf.create_ground_truth_model(model_name=model_name)
cell_release = mf.create_ground_truth_model(model_name=model_name, release=True)

# probe = mf.define_electrode(probe_type=probe_type)
probe=None

param_names = [param.name for param in cell.params.values() if not param.frozen]
sim = ephys.simulators.LFPySimulator(cvode_active=True, mechanisms_directory=model_folder)

params_release = {}
for param in cell_release.params_by_names(param_names):
    params_release[param.name] = param.value

# Test calcium spike

In [ ]:
# opt_results = {}
# for strategy in np.unique(df_model.strategy):
#     opt_results[strategy] = {}
#     opt_df = df_model.query(f"strategy == '{strategy}'")
#     best_idx = np.argmin(opt_df.best_fitness)
#     params_sample = opt_df.iloc[best_idx]
#     params_dict = {k: v for k, v in zip(param_names, params_sample.best_params)}
#     opt_results[strategy]["best_fitness"] = params_sample.best_fitness
#     opt_results[strategy]["best_params"] = params_dict
#     print(f"{strategy} --  best fitness: {params_sample.best_fitness}")

In [ ]:
bac_kwargs = {"syn_weight": 2, "pulse_amp": 1}

In [ ]:
print("Computing RELEASE")
responses_release = mf.utils.simulate_BAC_responses(cell, params_release, sim, **bac_kwargs)

bac_responses = {}
for strategy in np.unique(df_model.strategy):
    print(f"Computing {strategy}")
    best_params = opt_results_all[strategy]["best_params"]

    responses = mf.utils.simulate_BAC_responses(cell, best_params, sim, **bac_kwargs)
    bac_responses[strategy] = responses

In [ ]:
ax_dict = {}

loc_names = responses_release.keys()
for loc in responses_release:
    fig, ax = plt.subplots(figsize=figsize)
    ax_dict[loc] = ax
    
    resp = responses_release[loc]
    ax.plot(resp["time"], resp["voltage"], alpha=0.8, color="k", label="GT")
    
    for strategy in bac_responses:
        responses = bac_responses[strategy]
        resp = responses[loc]
        ax.plot(resp["time"], resp["voltage"], alpha=0.8, color=colors_dict[strategy], label=strategy.upper())
        
    ax.set_xlabel("time (ms)", fontsize=12)
    ax.set_ylabel("$V_m$ (mV)", fontsize=12)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.set_title(f"Recording from {loc}", fontsize=15)
#     ax.axvline(ms_before, color="gray", ls="--")
    ax.legend()
